In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import sklearn

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

import time

import matplotlib.pyplot as plt

import pickle

In [ ]:
data = pd.read_csv('cs-training-nonull.csv')

In [139]:
# global constants
N_ESTIMATORS = 1000
# numero di features compreso le due colonne indicatore aggiunte
N_FEATURES = 12

# nome della colonna con le label (da prevedere)
TARGET = 'SeriousDlqin2yrs'

# colonne con missing values
COL1_MISSING = 'MonthlyIncome'
COL2_MISSING = 'NumberOfDependents'

# nomi delle due colonne indicator (valgono 1 laddove il dato è inputato)
IND1 = 'isna_mi'
IND2 = 'isna_nod'

ind_col = [IND1, IND2]

# prese da stats sul train test, usate per inputare i missing values su COL1 e CL2
MONTHLY_INC_MEDIAN = 5400.0
N_OF_DEP_MODE = 0

In [140]:
ind = data[ind_col].values

In [141]:
train_df = data.drop(ind_col, axis = 1)

# estrae X: matrice features ed y, labels
y = data[TARGET].values
xtrain_df = train_df.drop(TARGET, axis = 1)
x_train = xtrain_df.values


In [142]:
# prepara lo scaling
scaler = StandardScaler()

# calcola i parametri di scaling solo sul train set
scaler.fit(x_train)

StandardScaler()

In [143]:
x_train_scaled = scaler.transform(x_train)

In [144]:
x_train_scaled = np.c_[x_train_scaled, ind]

In [145]:
assert x_train_scaled.shape[1] == N_FEATURES

In [146]:
param_dist = {'objective':'binary:logistic', 'n_estimators':N_ESTIMATORS,
             'learning_rate':0.01, 'max_depth': 5,
             'random_state': 0, 'min_child_weight': 10.0,
             'max_delta_step': 1.8, 'colsample_bytree': 0.4,
             'subsample': 0.8, 'eta': 0.1,
             'gamma': 0.1, 'num_boost_round' : 391,'booster':'gbtree','verbosity':1}

In [147]:
clfK = []

In [148]:
kf = KFold(n_splits=10)

In [149]:
for train_index, valid_index in kf.split(x_train_scaled):
    y_train = y[train_index] 
    y_valid = y[valid_index]
    print(y_train.shape,y_valid.shape)
    clf = xgb.XGBClassifier(use_label_encoder=False, **param_dist)
    clfK.append(clf.fit(x_train_scaled[train_index], y_train,
        eval_set=[(x_train_scaled[train_index], y_train), (x_train_scaled[valid_index], y_valid)],
        eval_metric='auc', verbose=True))

(131418,) (14603,)
[17:00:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { num_boost_round } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-auc:0.79825	validation_1-auc:0.79500
[1]	validation_0-auc:0.83192	validation_1-auc:0.81951
[2]	validation_0-auc:0.84292	validation_1-auc:0.83531
[3]	validation_0-auc:0.84971	validation_1-auc:0.84129
[4]	validation_0-auc:0.84968	validation_1-auc:0.84180
[5]	validation_0-auc:0.85688	validation_1-auc:0.84784
[6]	validation_0-auc:0.85747	validation_1-auc:0.84811
[7]	validation_0-auc:0.86100	validation_1-auc:0.85038
[8]	validation_0-auc:0.86104	validation_1-auc:0.85022
[9]	validation_0-auc:0.86259	validation_1-auc:0.85383
[10]	validation_0-auc:0.86294	validation_1-auc:0.85500
[11]	validation_0-auc

[142]	validation_0-auc:0.86864	validation_1-auc:0.85885
[143]	validation_0-auc:0.86862	validation_1-auc:0.85885
[144]	validation_0-auc:0.86861	validation_1-auc:0.85889
[145]	validation_0-auc:0.86862	validation_1-auc:0.85894
[146]	validation_0-auc:0.86867	validation_1-auc:0.85904
[147]	validation_0-auc:0.86866	validation_1-auc:0.85907
[148]	validation_0-auc:0.86870	validation_1-auc:0.85911
[149]	validation_0-auc:0.86876	validation_1-auc:0.85917
[150]	validation_0-auc:0.86879	validation_1-auc:0.85918
[151]	validation_0-auc:0.86887	validation_1-auc:0.85925
[152]	validation_0-auc:0.86887	validation_1-auc:0.85934
[153]	validation_0-auc:0.86889	validation_1-auc:0.85932
[154]	validation_0-auc:0.86885	validation_1-auc:0.85924
[155]	validation_0-auc:0.86887	validation_1-auc:0.85921
[156]	validation_0-auc:0.86891	validation_1-auc:0.85922
[157]	validation_0-auc:0.86891	validation_1-auc:0.85918
[158]	validation_0-auc:0.86890	validation_1-auc:0.85911
[159]	validation_0-auc:0.86893	validation_1-auc:

[289]	validation_0-auc:0.87063	validation_1-auc:0.85956
[290]	validation_0-auc:0.87064	validation_1-auc:0.85952
[291]	validation_0-auc:0.87065	validation_1-auc:0.85951
[292]	validation_0-auc:0.87066	validation_1-auc:0.85954
[293]	validation_0-auc:0.87069	validation_1-auc:0.85955
[294]	validation_0-auc:0.87067	validation_1-auc:0.85955
[295]	validation_0-auc:0.87069	validation_1-auc:0.85954
[296]	validation_0-auc:0.87075	validation_1-auc:0.85954
[297]	validation_0-auc:0.87078	validation_1-auc:0.85959
[298]	validation_0-auc:0.87081	validation_1-auc:0.85960
[299]	validation_0-auc:0.87085	validation_1-auc:0.85966
[300]	validation_0-auc:0.87087	validation_1-auc:0.85969
[301]	validation_0-auc:0.87088	validation_1-auc:0.85971
[302]	validation_0-auc:0.87091	validation_1-auc:0.85970
[303]	validation_0-auc:0.87094	validation_1-auc:0.85977
[304]	validation_0-auc:0.87096	validation_1-auc:0.85975
[305]	validation_0-auc:0.87097	validation_1-auc:0.85973
[306]	validation_0-auc:0.87097	validation_1-auc:

[436]	validation_0-auc:0.87273	validation_1-auc:0.86039
[437]	validation_0-auc:0.87274	validation_1-auc:0.86038
[438]	validation_0-auc:0.87275	validation_1-auc:0.86042
[439]	validation_0-auc:0.87275	validation_1-auc:0.86043
[440]	validation_0-auc:0.87277	validation_1-auc:0.86044
[441]	validation_0-auc:0.87278	validation_1-auc:0.86044
[442]	validation_0-auc:0.87278	validation_1-auc:0.86043
[443]	validation_0-auc:0.87279	validation_1-auc:0.86043
[444]	validation_0-auc:0.87281	validation_1-auc:0.86045
[445]	validation_0-auc:0.87283	validation_1-auc:0.86045
[446]	validation_0-auc:0.87287	validation_1-auc:0.86045
[447]	validation_0-auc:0.87286	validation_1-auc:0.86045
[448]	validation_0-auc:0.87289	validation_1-auc:0.86045
[449]	validation_0-auc:0.87291	validation_1-auc:0.86048
[450]	validation_0-auc:0.87291	validation_1-auc:0.86047
[451]	validation_0-auc:0.87292	validation_1-auc:0.86046
[452]	validation_0-auc:0.87294	validation_1-auc:0.86047
[453]	validation_0-auc:0.87296	validation_1-auc:

[583]	validation_0-auc:0.87445	validation_1-auc:0.86091
[584]	validation_0-auc:0.87447	validation_1-auc:0.86091
[585]	validation_0-auc:0.87448	validation_1-auc:0.86090
[586]	validation_0-auc:0.87449	validation_1-auc:0.86090
[587]	validation_0-auc:0.87450	validation_1-auc:0.86091
[588]	validation_0-auc:0.87452	validation_1-auc:0.86089
[589]	validation_0-auc:0.87453	validation_1-auc:0.86090
[590]	validation_0-auc:0.87454	validation_1-auc:0.86092
[591]	validation_0-auc:0.87456	validation_1-auc:0.86094
[592]	validation_0-auc:0.87456	validation_1-auc:0.86093
[593]	validation_0-auc:0.87458	validation_1-auc:0.86095
[594]	validation_0-auc:0.87460	validation_1-auc:0.86094
[595]	validation_0-auc:0.87461	validation_1-auc:0.86095
[596]	validation_0-auc:0.87462	validation_1-auc:0.86094
[597]	validation_0-auc:0.87463	validation_1-auc:0.86093
[598]	validation_0-auc:0.87465	validation_1-auc:0.86095
[599]	validation_0-auc:0.87467	validation_1-auc:0.86096
[600]	validation_0-auc:0.87469	validation_1-auc:

...

[355]	validation_0-auc:0.86937	validation_1-auc:0.87741
[356]	validation_0-auc:0.86937	validation_1-auc:0.87741
[357]	validation_0-auc:0.86936	validation_1-auc:0.87742
[358]	validation_0-auc:0.86935	validation_1-auc:0.87742
[359]	validation_0-auc:0.86938	validation_1-auc:0.87743
[360]	validation_0-auc:0.86939	validation_1-auc:0.87741
[361]	validation_0-auc:0.86939	validation_1-auc:0.87740
[362]	validation_0-auc:0.86938	validation_1-auc:0.87741
[363]	validation_0-auc:0.86942	validation_1-auc:0.87741
[364]	validation_0-auc:0.86944	validation_1-auc:0.87742
[365]	validation_0-auc:0.86944	validation_1-auc:0.87742
[366]	validation_0-auc:0.86947	validation_1-auc:0.87740
[367]	validation_0-auc:0.86949	validation_1-auc:0.87735
[368]	validation_0-auc:0.86951	validation_1-auc:0.87734
[369]	validation_0-auc:0.86952	validation_1-auc:0.87728
[370]	validation_0-auc:0.86955	validation_1-auc:0.87730
[371]	validation_0-auc:0.86954	validation_1-auc:0.87733
[372]	validation_0-auc:0.86953	validation_1-auc:

[502]	validation_0-auc:0.87130	validation_1-auc:0.87812
[503]	validation_0-auc:0.87132	validation_1-auc:0.87810
[504]	validation_0-auc:0.87133	validation_1-auc:0.87810
[505]	validation_0-auc:0.87133	validation_1-auc:0.87810
[506]	validation_0-auc:0.87133	validation_1-auc:0.87811
[507]	validation_0-auc:0.87134	validation_1-auc:0.87814
[508]	validation_0-auc:0.87135	validation_1-auc:0.87813
[509]	validation_0-auc:0.87137	validation_1-auc:0.87813
[510]	validation_0-auc:0.87137	validation_1-auc:0.87813
[511]	validation_0-auc:0.87138	validation_1-auc:0.87811
[512]	validation_0-auc:0.87140	validation_1-auc:0.87810
[513]	validation_0-auc:0.87140	validation_1-auc:0.87812
[514]	validation_0-auc:0.87140	validation_1-auc:0.87814
[515]	validation_0-auc:0.87140	validation_1-auc:0.87813
[516]	validation_0-auc:0.87141	validation_1-auc:0.87810
[517]	validation_0-auc:0.87142	validation_1-auc:0.87811
[518]	validation_0-auc:0.87144	validation_1-auc:0.87810
[519]	validation_0-auc:0.87145	validation_1-auc:

[649]	validation_0-auc:0.87308	validation_1-auc:0.87843
[650]	validation_0-auc:0.87310	validation_1-auc:0.87842
[651]	validation_0-auc:0.87311	validation_1-auc:0.87839
[652]	validation_0-auc:0.87313	validation_1-auc:0.87839
[653]	validation_0-auc:0.87313	validation_1-auc:0.87839
[654]	validation_0-auc:0.87315	validation_1-auc:0.87839
[655]	validation_0-auc:0.87316	validation_1-auc:0.87839
[656]	validation_0-auc:0.87318	validation_1-auc:0.87842
[657]	validation_0-auc:0.87319	validation_1-auc:0.87843
[658]	validation_0-auc:0.87320	validation_1-auc:0.87844
[659]	validation_0-auc:0.87321	validation_1-auc:0.87843
[660]	validation_0-auc:0.87322	validation_1-auc:0.87842
[661]	validation_0-auc:0.87323	validation_1-auc:0.87841
[662]	validation_0-auc:0.87325	validation_1-auc:0.87841
[663]	validation_0-auc:0.87325	validation_1-auc:0.87841
[664]	validation_0-auc:0.87326	validation_1-auc:0.87840
[665]	validation_0-auc:0.87328	validation_1-auc:0.87841
[666]	validation_0-auc:0.87328	validation_1-auc:

[796]	validation_0-auc:0.87462	validation_1-auc:0.87853
[797]	validation_0-auc:0.87464	validation_1-auc:0.87853
[798]	validation_0-auc:0.87465	validation_1-auc:0.87852
[799]	validation_0-auc:0.87467	validation_1-auc:0.87851
[800]	validation_0-auc:0.87468	validation_1-auc:0.87850
[801]	validation_0-auc:0.87470	validation_1-auc:0.87850
[802]	validation_0-auc:0.87471	validation_1-auc:0.87850
[803]	validation_0-auc:0.87471	validation_1-auc:0.87850
[804]	validation_0-auc:0.87473	validation_1-auc:0.87850
[805]	validation_0-auc:0.87474	validation_1-auc:0.87849
[806]	validation_0-auc:0.87476	validation_1-auc:0.87850
[807]	validation_0-auc:0.87477	validation_1-auc:0.87849
[808]	validation_0-auc:0.87478	validation_1-auc:0.87849
[809]	validation_0-auc:0.87479	validation_1-auc:0.87850
[810]	validation_0-auc:0.87480	validation_1-auc:0.87849
[811]	validation_0-auc:0.87482	validation_1-auc:0.87850
[812]	validation_0-auc:0.87484	validation_1-auc:0.87850
[813]	validation_0-auc:0.87485	validation_1-auc:

[943]	validation_0-auc:0.87599	validation_1-auc:0.87841
[944]	validation_0-auc:0.87602	validation_1-auc:0.87842
[945]	validation_0-auc:0.87602	validation_1-auc:0.87841
[946]	validation_0-auc:0.87602	validation_1-auc:0.87841
[947]	validation_0-auc:0.87603	validation_1-auc:0.87841
[948]	validation_0-auc:0.87604	validation_1-auc:0.87841
[949]	validation_0-auc:0.87604	validation_1-auc:0.87841
[950]	validation_0-auc:0.87605	validation_1-auc:0.87842
[951]	validation_0-auc:0.87605	validation_1-auc:0.87842
[952]	validation_0-auc:0.87607	validation_1-auc:0.87843
[953]	validation_0-auc:0.87607	validation_1-auc:0.87843
[954]	validation_0-auc:0.87608	validation_1-auc:0.87842
[955]	validation_0-auc:0.87608	validation_1-auc:0.87843
[956]	validation_0-auc:0.87609	validation_1-auc:0.87843
[957]	validation_0-auc:0.87609	validation_1-auc:0.87842
[958]	validation_0-auc:0.87610	validation_1-auc:0.87842
[959]	validation_0-auc:0.87611	validation_1-auc:0.87843
[960]	validation_0-auc:0.87613	validation_1-auc:

In [150]:
# predictions on test set
orig_test = pd.read_csv('cs-test.csv')

# inpute missing values, add the two indicator columns
orig_test['isna_mi'] = 0
orig_test.loc[orig_test[COL1_MISSING].isna(), 'isna_mi'] = 1
orig_test.loc[orig_test[COL1_MISSING].isna(), COL1_MISSING] = MONTHLY_INC_MEDIAN

orig_test['isna_nod'] = 0
orig_test.loc[orig_test[COL2_MISSING].isna(), 'isna_nod'] = 1
orig_test.loc[orig_test[COL2_MISSING].isna(), COL2_MISSING] = N_OF_DEP_MODE

In [151]:
ind_test = orig_test[ind_col].values

In [152]:
orig_test = orig_test.drop(ind_col, axis = 1)

In [153]:
ID_COL_NAME = 'Unnamed: 0'
xorig_test = orig_test.drop(ID_COL_NAME, axis = 1)
xorig_test = xorig_test.drop(TARGET, axis = 1)

x_test = xorig_test.values

In [154]:
# aggiungi qui lo scaling !!!
x_test_scaled = scaler.transform(x_test)
# riaggiunge le colonne indicatore
x_test_scaled = np.c_[x_test_scaled, ind_test]

assert x_test_scaled.shape[1] == N_FEATURES

In [155]:
preds = []

In [156]:
for elem in clfK:
    preds.append(elem.predict_proba(x_test_scaled))
preds

[array([[0.935462  , 0.06453798],
        [0.9434073 , 0.05659272],
        [0.9834302 , 0.01656981],
        ...,
        [0.9941584 , 0.00584161],
        [0.92510563, 0.07489435],
        [0.95196754, 0.04803247]], dtype=float32),
 array([[0.9369868 , 0.06301321],
        [0.9438968 , 0.05610321],
        [0.9821551 , 0.01784491],
        ...,
        [0.99348277, 0.00651722],
        [0.92680794, 0.07319206],
        [0.95432216, 0.04567786]], dtype=float32),
 array([[0.937516  , 0.06248404],
        [0.9506397 , 0.04936028],
        [0.9812681 , 0.0187319 ],
        ...,
        [0.99428606, 0.00571393],
        [0.93795913, 0.06204085],
        [0.9581258 , 0.0418742 ]], dtype=float32),
 array([[0.936002  , 0.06399799],
        [0.9478916 , 0.05210844],
        [0.98211837, 0.0178816 ],
        ...,
        [0.9941065 , 0.00589356],
        [0.9358131 , 0.06418689],
        [0.95509547, 0.04490454]], dtype=float32),
 array([[0.9373944 , 0.06260562],
        [0.94252515, 0.0574748

In [157]:
y_pred = np.mean(preds,axis = 0)

In [158]:
y_pred

array([[0.93707   , 0.06293   ],
       [0.9461204 , 0.05387965],
       [0.98223746, 0.01776255],
       ...,
       [0.99402446, 0.00597543],
       [0.93353736, 0.0664625 ],
       [0.9549922 , 0.04500784]], dtype=float32)

In [159]:
y_pred.shape

(101503, 2)

In [160]:
y_pred = y_pred[:, 1]

In [161]:
# prepara il csv per la submission
result_dict = {"Id": orig_test[ID_COL_NAME].values,
              'Probability': y_pred}

FILE_SUB = 'submission20.csv'

# build a dataframe and save to csv
result_df = pd.DataFrame(result_dict)

result_df.to_csv(FILE_SUB, index=False, float_format='%.5f')